Import necessary libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Now let's take load the data and take a look at it.

In [2]:
df_observations = pd.read_csv("observations.csv")
df_species = pd.read_csv("species_info.csv")

In [3]:
print(df_observations)

                               scientific_name  \
0                           Vicia benghalensis   
1                               Neovison vison   
2                            Prunus subcordata   
3                         Abutilon theophrasti   
4                     Githopsis specularioides   
...                                        ...   
23291                     Croton monanthogynus   
23292                 Otospermophilus beecheyi   
23293  Heterotheca sessiliflora ssp. echioides   
23294                     Dicranella rufescens   
23295                           Cucurbita pepo   

                                 park_name  observations  
0      Great Smoky Mountains National Park            68  
1      Great Smoky Mountains National Park            77  
2                   Yosemite National Park           138  
3                      Bryce National Park            84  
4      Great Smoky Mountains National Park            85  
...                                    ...   

In [4]:
print(df_species)

            category                scientific_name  \
0             Mammal  Clethrionomys gapperi gapperi   
1             Mammal                      Bos bison   
2             Mammal                     Bos taurus   
3             Mammal                     Ovis aries   
4             Mammal                 Cervus elaphus   
...              ...                            ...   
5819  Vascular Plant               Solanum parishii   
5820  Vascular Plant                  Solanum xanti   
5821  Vascular Plant         Parthenocissus vitacea   
5822  Vascular Plant              Vitis californica   
5823  Vascular Plant            Tribulus terrestris   

                                           common_names conservation_status  
0                              Gapper's Red-Backed Vole                 NaN  
1                                 American Bison, Bison                 NaN  
2     Aurochs, Aurochs, Domestic Cattle (Feral), Dom...                 NaN  
3     Domestic Sheep, Moufl

So, in observations.csv we have a list of the number of times each species was observed in each park. In species_info.csv we have a list of species with their respective category, common name, and conservation status. What can we do with this? One thing we could do is make bar charts showing how many of each category are seen in each park. This would tell us immediately which parks have the most sightings and what type of wildlife is seen most often. Another thing we could do is look at endangered species and see if they are observed at significantly lower rates than other species in their category.

First, I'll write a function that gives me the category when I input the scientific name of a species. The simplest thing will be to just create a dictionary.

In [7]:
category_dict = dict(zip(df_species.scientific_name, df_species.category))
#print(category_dict)

Now I'll use it to make a new column.

In [8]:
getCategory = lambda scientific_name: category_dict[scientific_name]
df_observations['Category'] = df_observations.scientific_name.apply(getCategory)
print(df_observations)

                               scientific_name  \
0                           Vicia benghalensis   
1                               Neovison vison   
2                            Prunus subcordata   
3                         Abutilon theophrasti   
4                     Githopsis specularioides   
...                                        ...   
23291                     Croton monanthogynus   
23292                 Otospermophilus beecheyi   
23293  Heterotheca sessiliflora ssp. echioides   
23294                     Dicranella rufescens   
23295                           Cucurbita pepo   

                                 park_name  observations           Category  
0      Great Smoky Mountains National Park            68     Vascular Plant  
1      Great Smoky Mountains National Park            77             Mammal  
2                   Yosemite National Park           138     Vascular Plant  
3                      Bryce National Park            84     Vascular Plant  
4      Gr